In [ ]:
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import gc

In [ ]:
model_try = str(6)

In [ ]:
IMGS = np.load('/content/drive/MyDrive/YANG/npy/new/img_n_cropfix.npy')
TARGET = np.load('/content/drive/MyDrive/YANG/npy/new/label_cropfix.npy')

print("IMGS shape:", IMGS.shape)
print("TARGET shape:", TARGET.shape)
label_encoder = LabelEncoder()
t_encoded = label_encoder.fit_transform(TARGET)
target_tmp = to_categorical(t_encoded)

# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(IMGS, target_tmp, test_size=0.1,stratify=t_encoded, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(IMGS, t_encoded, test_size=0.1,stratify=t_encoded, random_state=42)

# 데이터셋 확인
print("Training set size:", X_train.shape)
print("y_train size:", y_train.shape)

IMGS shape: (18260, 224, 224, 3)
TARGET shape: (18260,)
Training set size: (16434, 224, 224, 3)
y_train size: (16434, 5)


In [ ]:
y_train

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
# 모델 설계
import tensorflow as tf
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


# 사전 학습된 ResNet50 모델 로드 (최상위 레이어 제외)
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
# model.add(Dense(1, activation='softmax'))

# 일부 층은 훈련되지 않도록 설정 (동결)
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam', #sgd, Nadam, Adagrad
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# model.compile(optimizer='adam', # sgd, Nadam, Adagrad
#               loss='mean_squared_error',
#               metrics=['mean_absolute_error'])

model.summary()

234545216/234545216 [==============================] - 12s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 7, 7, 2048)        58331648  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)              

In [ ]:
# 옵션 지정

os.makedirs(f"/content/drive/MyDrive/YANG/model/20240613_model4/m{model_try}", exist_ok=True)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
checkpoint = ModelCheckpoint(
    "/content/drive/MyDrive/YANG/model/20240613_model4/m"+model_try+"/model"+model_try+"_{val_accuracy:.2f}.keras",
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='auto')


# early_stopping = EarlyStopping(monitor='val_mean_absolute_error', patience=15, restore_best_weights=True)
# checkpoint = ModelCheckpoint(
#     "/content/drive/MyDrive/YANG/model/20240613_model4/m"+model_try+"/model"+model_try+"_{val_mean_absolute_error:.2f}.keras",
#     monitor='val_mean_absolute_error',
#     verbose=1,
#     save_best_only=True,
#     mode='auto')

In [ ]:
# 모델 학습
history=model.fit(X_train, y_train,
                  epochs=100,
                  batch_size=16,
                  callbacks=[checkpoint, early_stopping],
                  validation_split=0.2)

Epoch 1/100
822/822 [==============================] - ETA: 0s - loss: 1.3984 - accuracy: 0.3662
Epoch 1: val_accuracy improved from -inf to 0.48890, saving model to /content/drive/MyDrive/YANG/model/20240613_model4/m6/model6_0.49.keras
822/822 [==============================] - 76s 76ms/step - loss: 1.3984 - accuracy: 0.3662 - val_loss: 1.1540 - val_accuracy: 0.4889
Epoch 2/100
821/822 [============================>.] - ETA: 0s - loss: 1.1716 - accuracy: 0.4706
Epoch 2: val_accuracy improved from 0.48890 to 0.52936, saving model to /content/drive/MyDrive/YANG/model/20240613_model4/m6/model6_0.53.keras
822/822 [==============================] - 57s 70ms/step - loss: 1.1718 - accuracy: 0.4704 - val_loss: 1.0515 - val_accuracy: 0.5294
Epoch 3/100
821/822 [============================>.] - ETA: 0s - loss: 1.0458 - accuracy: 0.5333
Epoch 3: val_accuracy improved from 0.52936 to 0.58229, saving model to /content/drive/MyDrive/YANG/model/20240613_model4/m6/model6_0.58.keras
822/822 [========

In [ ]:
model.evaluate(X_test, y_test)

58/58 [==============================] - 7s 108ms/step - loss: 3.0005 - mean_absolute_error: 1.4003


[3.0005476474761963, 1.4003286361694336]

In [ ]:
aa = model.predict(X_test)

58/58 [==============================] - 6s 101ms/step
